In [0]:
! nvidia-smi

Fri May  3 04:11:09 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    33W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print('success!')

success!


In [0]:
from progressbar import ProgressBar, Percentage, Bar
from google.colab import files
import torch, pickle, os, sys, random, time, math, copy
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from queue import PriorityQueue

########################### ENTIRE DATASET ###########################
# german = drive.CreateFile({'id': '1I0_go5RhzEg2CkbpZgx8a6h4hA9qNeyY'})
# german.GetContentFile('./german_no_pad_sorted.pickle') 
# with open('./german_no_pad_sorted.pickle', 'rb') as f_in:
#     german = pickle.load(f_in)
    
# english = drive.CreateFile({'id': '1Tb0aYVfkj3YyXXTxWRkvDO7T__ovng1Q'})
# english.GetContentFile('./english_no_pad_sorted.pickle') 
# with open('./english_no_pad_sorted.pickle', 'rb') as f_in:
#     english = pickle.load(f_in)    

########################### 5K DATASET ###########################   
# german = drive.CreateFile({'id': '10epaM5VzSskc0xkriD0YN_a0Mg3-o1bg'})
# german.GetContentFile('./german_no_pad_sorted_5k.pickle') 
# with open('./german_no_pad_sorted_5k.pickle', 'rb') as f_in:
#     german = pickle.load(f_in)

# english = drive.CreateFile({'id': '15P0N9sLAIPyxF0Uhgx-01P2s-xOLp6xb'})
# english.GetContentFile('./english_no_pad_sorted_5k.pickle') 
# with open('./english_no_pad_sorted_5k.pickle', 'rb') as f_in:
#     english = pickle.load(f_in)

########################### 50K DATASET ###########################    
# german = drive.CreateFile({'id': '1bNBrVR9TCNXRqlS9kxPa3EffYyZnAfCw'})
# german.GetContentFile('./german_no_pad_sorted_50k.pickle') 
# with open('./german_no_pad_sorted_50k.pickle', 'rb') as f_in:
#     german = pickle.load(f_in)
    
# english = drive.CreateFile({'id': '1XnCWgSLGcp3lCdUK0ciS5m6mwI5L5R2l'})
# english.GetContentFile('./english_no_pad_sorted_50k.pickle') 
# with open('./english_no_pad_sorted_50k.pickle', 'rb') as f_in:
#     english = pickle.load(f_in)

# ########################### ENTIRE DATASET (WITH UNK) ###########################
# german = drive.CreateFile({'id': '1FuoqjfMofNfkIRXBIWDU_cIVV9BBWcs-'})
# german.GetContentFile('./german_unk.pickle') 
# with open('./german_unk.pickle', 'rb') as f_in:
#     german = pickle.load(f_in)
    
# english = drive.CreateFile({'id': '18Q-WRZ2JBwkAJLXwiVuWPfyE-0k76FDA'})
# english.GetContentFile('./english_unk.pickle') 
# with open('./english_unk.pickle', 'rb') as f_in:
#     english = pickle.load(f_in)    
    
########################### ENTIRE DATASET (WITH BPE) ###########################
# german = drive.CreateFile({'id': '1KNeZ_WQPUudwlJsg5_kUqPlLjJGBynsn'})
# german.GetContentFile('./german_bpe.pickle') 
# with open('./german_bpe.pickle', 'rb') as f_in:
#     german = pickle.load(f_in)
    
# english = drive.CreateFile({'id': '1ObXLocsuZsVGH3MVxSUdDBpiMlgBqVP7'})
# english.GetContentFile('./english_bpe.pickle') 
# with open('./english_bpe.pickle', 'rb') as f_in:
#     english = pickle.load(f_in)     

########################### ENTIRE DATASET (WITH BPE) ###########################
german = drive.CreateFile({'id': '19_OtUbvPMe7pkhnljnETw2tmyECfBNEu'})
german.GetContentFile('./german_bpe_40.pickle') 
with open('./german_bpe.pickle', 'rb') as f_in:
    german = pickle.load(f_in)
    
english = drive.CreateFile({'id': '1KK9QB8Y5btOKh-tH9ynR3sUtqzwz2U2E'})
english.GetContentFile('./english_bpe_40.pickle') 
with open('./english_bpe.pickle', 'rb') as f_in:
    english = pickle.load(f_in)     


# for i in range(len(german['train'])):
#     german['train'][i] = torch.LongTensor(german['train'][i]).cuda()
#     english['train'][i] = torch.LongTensor(english['train'][i]).cuda()
    
# for i in range(len(german['dev'])):
#     german['dev'][i] = torch.LongTensor(german['dev'][i]).cuda()
#     english['dev'][i] = torch.LongTensor(english['dev'][i]).cuda()

training_data = [[german['train'][i], english['train'][i]] for i in range(len(german['train']))]
validation_data = [[german['dev'][i], english['dev'][i]] for i in range(len(german['dev']))]

max_len_train = len(max(german['train'], key=len))
max_len_valid = len(max(german['dev'], key=len))

print(max_len_train, max_len_valid)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

906 281
cuda:0


In [0]:
def to_padded_tensor(batch):
    max_len_src = max([len(sent['source']) for sent in batch])  
    max_len_trg = max([len(sent['target']) for sent in batch])

    for sent in batch:
        dif_src = max_len_src - len(sent['source'])
        dif_trg = max_len_trg - len(sent['target'])

        if dif_src > 0:
            pad_list_src = [0 for d in range(dif_src)]
            sent['source'].extend(pad_list_src)

        if dif_trg > 0:
            pad_list_trg = [0 for d in range(dif_trg)]
            sent['target'].extend(pad_list_trg)       
    
    source_sent_len = max_len_src
    target_sent_len = max_len_trg
    
    batch_size = len(batch)
    
    source = torch.empty((source_sent_len, batch_size)).long().cpu()    
    target = torch.empty((target_sent_len, batch_size)).long().cpu()
    
#     print(batch[0]['target'])
    
    for i in range(len(batch)):
        source[:,i] = torch.tensor(batch[i]['source'])
        target[:,i] = torch.tensor(batch[i]['target'])
        
    padded_tensor = {"source": source.to(device),
                    "target": target.to(device)}
    
    return padded_tensor

In [0]:
def bake_batches(de, en, batch_size=1700, min_len=3, max_len=768, bucket_step=3):
    german = copy.deepcopy(de)
    english = copy.deepcopy(en)
    
    buckets = [[] for i in range(0, max_len, bucket_step)]
    bucket_lengths = [0 for i in buckets]
    batches = []
    
    # For every sentence in the dataset, find its corresponding bucket and put it in there, once the bucket
    # hits the batch size, ship it off to the batches list
    for i in range(len(german)):
        sent2sent = {"source": german[i],
                     "target": english[i]}
        
        # calculate the index of the buckets to put the sentence into, = len(Sentence) // Bucket_step - 1
        idx = len(sent2sent['source'])//bucket_step - 1
        
        if bucket_lengths[idx] + len(sent2sent['source']) > batch_size:
            batches.append(to_padded_tensor(buckets[idx][:]))
            del buckets[idx][:]
            buckets[idx].append(sent2sent)
            bucket_lengths[idx] = len(sent2sent['source'])
        else:
            buckets[idx].append(sent2sent)
            bucket_lengths[idx] += len(sent2sent['source'])
            

    # for any remaining buckets that did not get sent off, send them off to batches
    for b in buckets:
        if b: # if the list has any value in it
            batches.append(to_padded_tensor(b[:]))
            del b[:]
    
    return batches

In [0]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src sent len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src sent len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src sent len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src sent len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

In [0]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Parameter(torch.rand(dec_hid_dim))
        
    def forward(self, hidden, encoder_outputs, beam=False):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat encoder hidden state src_len times
        if not beam:
            hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        if not self.training and beam:
            hidden = hidden.repeat(encoder_outputs.size(0),encoder_outputs.size(1), 1)
        
        #hidden = [batch size, src sent len, dec hid dim]
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src sent len, dec hid dim]
        
        energy = energy.permute(0, 2, 1)
        
        #energy = [batch size, dec hid dim, src sent len]
        
        #v = [dec hid dim]
        
        v = self.v.repeat(batch_size, 1).unsqueeze(1)
        
        #v = [batch size, 1, dec hid dim]
                
        attention = torch.bmm(v, energy).squeeze(1)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

In [0]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs, beam=False):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        if not beam: 
            input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [sent len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #sent len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        output = self.out(torch.cat((output, weighted, embedded), dim = 1))
        
        #output = [bsz, output dim]
        
        return output, hidden.squeeze(0)

In [0]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.softmax = nn.LogSoftmax()
        
    def beam_search(self, target_tensor, decoder_hiddens, encoder_outputs=None):
        '''
        :param target_tensor: target indexes tensor of shape [B, T] where B is the batch size and T is the maximum length of the output sentence
        :param decoder_hidden: input tensor of shape [1, B, H] for start of the decoding
        :param encoder_outputs: if you are using attention mechanism you can pass encoder outputs, [T, B, H] where T is the maximum length of input sentence
        :return: decoded_batch
        '''

        beam_width = 4
        topk = 4 # how many sentence do you want to generate
        decoded_batch = []
        # decoding goes sentence by sentence
        sentence_length, num_sentences = target_tensor.size()
        for idx in range(num_sentences):
            if isinstance(decoder_hiddens, tuple):  # LSTM case
                decoder_hidden = (decoder_hiddens[0][:,idx, :].unsqueeze(0),decoder_hiddens[1][:,idx, :].unsqueeze(0))
            else:
                decoder_hidden = decoder_hiddens[idx, :].unsqueeze(0)
            encoder_output = encoder_outputs[:,idx, :].unsqueeze(1)

            # Start with the start of the sentence token
            decoder_input = torch.LongTensor([[2]]).cuda()

            # Number of sentence to generate
            endnodes = []
            number_required = min((topk + 1), topk - len(endnodes))

            # starting node -  hidden vector, previous node, word id, logp, length
            node = BeamSearchNode(decoder_hidden, None, decoder_input, 0, 1)
            nodes = PriorityQueue()

            # start the queue
            nodes.put(node)
            qsize = 1

            # start beam search
            while True:
                # give up when decoding takes too long
                if qsize > 10000: break

                # fetch the best node
                n = nodes.get()
                decoder_input = n.wordid
                decoder_hidden = n.h

                if n.wordid.item() == 3 and n.prevNode != None:
                    endnodes.append(n)
                    # if we reached maximum # of sentences required
                    if len(endnodes) >= number_required:
                        break
                    else:
                        continue
                
                # decode for one step using decoder
                decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_output, True)

                # PUT HERE REAL BEAM SEARCH OF TOP
                # take softmax first, then log
                decoder_output = self.softmax(decoder_output)
                
                log_prob, indexes = torch.topk(decoder_output, beam_width)
                nextnodes = []

                for new_k in range(beam_width):
                    decoded_t = indexes[0][new_k].view(1, -1)
                    log_p = log_prob[0][new_k].item()

                    node = BeamSearchNode(decoder_hidden, n, decoded_t, n.logp - log_p, n.leng + 1)
                    score = -node.eval()
                    nextnodes.append(node)

                # put them into queue
                for i in range(len(nextnodes)):
                    nn = nextnodes[i]
                    nodes.put(nn)
                    # increase qsize
                qsize += len(nextnodes) - 1

            # choose nbest paths, back trace them
            if len(endnodes) == 0:
                endnodes = [nodes.get() for _ in range(topk)]

            utterances = []
            max_prob = -100000
            max_sen = False
            final_sentence = []
            for n in sorted(endnodes):
                utterance = []
                if max_prob < n.logp:
                    max_prob = n.logp
                    max_sen = True
                utterance.append(n.wordid)
                # back trace
                while n.prevNode != None:
                    n = n.prevNode
                    utterance.append(n.wordid)
                  
                utterance = utterance[::-1]
                if max_sen:
                    max_sen = False
                    final_sentence = utterance
                utterances.append(utterance)
                
                
            decoded_batch.append(torch.IntTensor(final_sentence))

        return decoded_batch
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5, run_beam=False):
        
        #src = [src sent len, batch size]
        #trg = [trg sent len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        output = trg[0,:]
        
        if run_beam:
            temp = self.beam_search(trg, hidden ,encoder_outputs)
            return temp
        # else use greedy
        
        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)

        return outputs

In [0]:
class BeamSearchNode(object):
    def __init__(self, hiddenstate, previousNode, wordId, logProb, length):
        '''
        :param hiddenstate:
        :param previousNode:
        :param wordId:
        :param logProb:
        :param length:
        '''
        self.h = hiddenstate
        self.prevNode = previousNode
        self.wordid = wordId
        self.logp = logProb
        self.leng = length

    def eval(self, alpha=1.0):
        reward = 0
        # Add here a function for shaping a reward

        return self.logp / float(self.leng - 1 + 1e-6) + alpha * reward
      
      
    def __lt__(self, other):
        return self.logp < other.logp

In [0]:
def train(model, batches, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, b in enumerate(batches):
        
        source = b['source']
        target = b['target']
        
        optimizer.zero_grad()
        
        output = model(source, target)
        
        if i%(len(batches)//4) == 0: 
            print( "batch:", i, ",", torch.cuda.memory_allocated(device)/1e6, "MB used") 
        
        #trg = [trg sent len, batch size]
        #output = [trg sent len, batch size, output dim]
        
        output = output[1:].view(-1, output.shape[-1])
        target = target[1:].view(-1)
        
        #trg = [(trg sent len - 1) * batch size]
        #output = [(trg sent len - 1) * batch size, output dim]
        
        loss = criterion(output, target)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(batches)

In [0]:
def evaluate(model, batches, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, b in enumerate(batches):
            
            source = b['source']
            target = b['target']

            output = model(source, target, 0) #turn off teacher forcing

            #trg = [trg sent len, batch size]
            #output = [trg sent len, batch size, output dim]

            output = output[1:].view(-1, output.shape[-1])
            target = target[1:].view(-1)

            #trg = [(trg sent len - 1) * batch size]
            #output = [(trg sent len - 1) * batch size, output dim]

            loss = criterion(output, target)

            epoch_loss += loss.item()
        
    return epoch_loss / len(batches)

In [0]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

In [0]:
def average_sentence_size(german, desired_batch_size=128):
    summation = 0
    for i in german['train']:
        summation += len(i)
    avg_tok_sent = summation/len(german['train'])
    
    print("To get an average batch size of", desired_batch_size, "Use a batch_size value of:", int(desired_batch_size*avg_tok_sent))
    
average_sentence_size(german, desired_batch_size=128)

To get an average batch size of 128 Use a batch_size value of: 3462


In [0]:
N_EPOCHS = 20
CLIP = 1

INPUT_DIM = len(german['idx2word'])
OUTPUT_DIM = len(english['idx2word'])
ENC_DROPOUT = 0.25
DEC_DROPOUT = 0.4
ENC_EMB_DIM = 500
DEC_EMB_DIM = 500
ENC_HID_DIM = 700
DEC_HID_DIM = 700
    
attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

model.apply(init_weights)

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index = 0)

best_valid_loss = float('inf')
bad_epoch_cnt = 0

results = {"hyperparams": (ENC_EMB_DIM, DEC_HID_DIM),
           "train_loss": [],
           "valid_loss": [],
           "train_ppl": [],
           "valid_ppl": []}

valid_batches = bake_batches(german['dev'], english['dev'], batch_size=3400, max_len=max_len_valid)

for epoch in range(N_EPOCHS):
    random.shuffle(training_data)
    random.shuffle(validation_data)

    de_shuffled_td = [td[0] for td in training_data]
    en_shuffled_td = [td[1] for td in training_data]

    print("Baking batches...")
    train_batches = bake_batches(de_shuffled_td, en_shuffled_td, batch_size=2400, max_len=max_len_train)
    print("Done.")

    start_time = time.time()

    train_loss = train(model, train_batches, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_batches, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        bad_epoch_cnt = 0
        torch.save(model.state_dict(), 'best-model.pt')
    else:
        bad_epoch_cnt += 1  
        
    print("Validation loss has not improved in", bad_epoch_cnt, "epochs")        
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    
    results['train_loss'].append(train_loss)
    results['valid_loss'].append(valid_loss)
    results['train_ppl'].append(math.exp(train_loss))
    results['valid_ppl'].append(math.exp(valid_loss))
    
    if bad_epoch_cnt >= 8:
        print("Early Stopping")
        break

Baking batches...
Done.
batch: 0 , 1858.74176 MB used
batch: 579 , 2701.844992 MB used
batch: 1158 , 3121.271296 MB used
batch: 1737 , 3299.498496 MB used
batch: 2316 , 7497.864704 MB used
Validation loss has not improved in 0 epochs
Epoch: 01 | Time: 41m 23s
	Train Loss: 4.772 | Train PPL: 118.150
	 Val. Loss: 5.679 |  Val. PPL: 292.733
Baking batches...
Done.
batch: 0 , 2497.376256 MB used
batch: 580 , 3268.04224 MB used
batch: 1160 , 3545.936384 MB used
batch: 1740 , 2751.855104 MB used
Validation loss has not improved in 0 epochs
Epoch: 02 | Time: 41m 20s
	Train Loss: 3.735 | Train PPL:  41.892
	 Val. Loss: 5.505 |  Val. PPL: 245.902
Baking batches...
Done.
batch: 0 , 2777.792512 MB used
batch: 579 , 2913.929216 MB used
batch: 1158 , 5013.400064 MB used
batch: 1737 , 2420.148736 MB used
batch: 2316 , 7517.376512 MB used
Validation loss has not improved in 0 epochs
Epoch: 03 | Time: 41m 20s
	Train Loss: 3.403 | Train PPL:  30.058
	 Val. Loss: 5.348 |  Val. PPL: 210.286
Baking batche

In [0]:
results_test_a = {}
results_test_b = {}

results_test_a['train_loss'] = [4.951, 4.208, 3.919, 3.738, 3.631, 3.566, 3.511     ]
results_test_a['valid_loss'] = [5.028, 4.961, 5.049, 5.060, 5.087, 5.102, 5.142    ]
results_test_a['train_ppl'] = [141.283, 67.240, 50.341, 42.028, 37.761, 35.362, 33.467]
results_test_a['valid_ppl'] = [152.631, 142.753, 155.863, 157.552, 161.846, 164.395, 171.068]

results_test_b['train_loss'] = [4.945,  4.226, 3.940, 3.759, 3.651, 3.587, 3.536]
results_test_b['valid_loss'] = [5.310, 5.215, 5.187, 5.247, 5.323, 5.303, 5.368]
results_test_b['train_ppl'] = [140.527,  68.470, 51.401, 42.904, 38.516, 36.116, 34.334]
results_test_b['valid_ppl'] = [202.433, 183.977, 178.941, 189.977, 204.979, 200.840, 214.369]

In [0]:
def sample_net(net, batches):    
    preds = []
    targets = []
    sources = []
    sampled_batches = {}
    
       
    with torch.no_grad():
        for i, b in enumerate(batches):
            print(i, "out of", len(batches))
            source = b['source']
            target = b['target']
            
#             print("L length x N sentences:", target.shape)
            pred = net.forward(source, target, teacher_forcing_ratio=0, run_beam=False)
#             print(type(output))
#             print("L length x N sentences x V vocabulary:", pred.shape)
            pred = torch.argmax(pred, dim=2)
            
#             print("L length x N sentences:", pred.shape)
#             print("L length x N sentences:", target.shape)
            
            preds.append(pred)
            targets.append(target)
            sources.append(source)
    
    sampled_batches['source'] = sources
    sampled_batches['target'] = targets
    sampled_batches['prediction'] = preds
    
    return sampled_batches
CLIP = 1

INPUT_DIM = len(german['idx2word'])
OUTPUT_DIM = len(english['idx2word'])
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)
model.load_state_dict(torch.load("best-model.pt"))

batches = bake_batches(german['dev'], english['dev'])
sampled_batches = sample_net(model, batches)


0 out of 196
1 out of 196
2 out of 196
3 out of 196
4 out of 196
5 out of 196
6 out of 196
7 out of 196
8 out of 196
9 out of 196
10 out of 196
11 out of 196
12 out of 196
13 out of 196
14 out of 196
15 out of 196
16 out of 196
17 out of 196
18 out of 196
19 out of 196
20 out of 196
21 out of 196
22 out of 196
23 out of 196
24 out of 196
25 out of 196
26 out of 196
27 out of 196
28 out of 196
29 out of 196
30 out of 196
31 out of 196
32 out of 196
33 out of 196
34 out of 196
35 out of 196
36 out of 196
37 out of 196
38 out of 196
39 out of 196
40 out of 196
41 out of 196
42 out of 196
43 out of 196
44 out of 196
45 out of 196
46 out of 196
47 out of 196
48 out of 196
49 out of 196
50 out of 196
51 out of 196
52 out of 196
53 out of 196
54 out of 196
55 out of 196
56 out of 196
57 out of 196
58 out of 196
59 out of 196
60 out of 196
61 out of 196
62 out of 196
63 out of 196
64 out of 196
65 out of 196
66 out of 196
67 out of 196
68 out of 196
69 out of 196
70 out of 196
71 out of 196
72

In [0]:
! pip install -U sacremoses

     |████████████████████████████████| 122kB 42.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/af/c2/17/52c81a9182b3c89376f28aa1500a1088377a37951ed2d78335
Successfully built sacremoses


In [0]:
from sacremoses import MosesDetokenizer

In [0]:
def print_samples(batches):
    # Each of the inputs should be a tensor batch of shape L x N
    # print(sources.shape)
    source = batches['source']
    target = batches['target']
    prediction = batches['prediction']
    
    sources_tokenized = []
    targets_tokenized = []
    preds_tokenized = []
    
#     md_de = MosesDetokenizer(lang='en')
#     md_en = MosesDetokenizer(lang='en')


    for b in range(len(batches['source'])):
        for sent in range(source[b].shape[1]):
            w = 1
            src_sent = []
            while w < target[b][:,sent].shape[0] and target[b][w,sent].item()!=3:
                idx = target[b][w,sent].item()
                src_sent.append(german['idx2word'][idx])
                w+=1
            sources_tokenized.append(src_sent)
            
            w = 1
            tgt_sent = []
            while w < target[b][:,sent].shape[0] and target[b][w,sent].item()!=3:
                idx = target[b][w,sent].item()
                tgt_sent.append(english['idx2word'][idx])
                w+=1
            targets_tokenized.append(tgt_sent)

            w = 1
            pred_sent = []
            while w < prediction[b][:,sent].shape[0] and prediction[b][w,sent].item()!=3:
                idx = prediction[b][w,sent].item()
                pred_sent.append(english['idx2word'][idx])
                w+=1
            preds_tokenized.append(pred_sent)
    
    with open('source.out', 'w') as f:
        for sent in sources_tokenized:
            x = " ".join(sent)
            f.write(x + '\n')
    
    with open('target.out', 'w') as f:
        for sent in targets_tokenized:
            x = " ".join(sent)
            f.write(x + '\n')
            
    with open('pred.out', 'w') as f:
        for sent in preds_tokenized:
            y = " ".join(sent)
            f.write(y + '\n')
            
print_samples(sampled_batches)

In [0]:
!sed -r -i 's/(@@ )|(@@ ?$)//g' pred.out
!sed -r -i 's/(@@ )|(@@ ?$)//g' source.out
!sed -r -i 's/(@@ )|(@@ ?$)//g' target.out

In [0]:
!git clone https://github.com/moses-smt/mosesdecoder.git
!pip install sacrebleu

Cloning into 'mosesdecoder'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 147302 (delta 0), reused 0 (delta 0), pack-reused 147295
Receiving objects: 100% (147302/147302), 129.70 MiB | 19.38 MiB/s, done.
Resolving deltas: 100% (113833/113833), done.
  Stored in directory: /root/.cache/pip/wheels/40/f1/a9/4cce0ec602e8d195da27bb9b8f6708ec778fbafdbabb097fde
Successfully built sacrebleu


In [0]:
%%shell
#!/bin/bash

# This is a reference to the gold translations from the dev set
REFERENCE_FILE="target.out"

# XXX: Change the following line to point to your model's output!
TRANSLATED_FILE="pred.out"

# The model output is expected to be in a tokenized form. Note, that if you
# tokenized your inputs to the model, then simply joined each output token with
# whitespace you should get tokenized outputs from your model.
# i.e. each output token is separate by whitespace
# e.g. "My model 's output is interesting ."
perl "mosesdecoder/scripts/tokenizer/detokenizer.perl" -l en < "$TRANSLATED_FILE" > "$TRANSLATED_FILE.detok"

PARAMS=("-tok" "intl" "-l" "de-en" "$REFERENCE_FILE")
sacrebleu "${PARAMS[@]}" < "$TRANSLATED_FILE.detok"

Detokenizer Version $Revision: 4134 $
Language: en
BLEU+case.mixed+lang.de-en+numrefs.1+smooth.exp+tok.intl+version.1.3.2 = 11.4 43.4/16.7/7.3/3.2 (BP = 1.000 ratio = 1.001 hyp_len = 171217 ref_len = 171009)
